In [ ]:
#| default_exp datascience
from nbdev import *
from nbdev.showdoc import *


# Advent of Code Utils

> A collection of somewhat handy functions to make your AoC puzzle life solving a bit easier

In [ ]:
#| exporti
from collections.abc import Iterable
from collections import namedtuple, deque
import contextlib
from functools import reduce
import hashlib
import heapq
import logging
from math import sqrt, gcd
from pathlib import Path
import time
import pickle
import pandas as pd
import numpy as np


## Data science helpers

In [ ]:
#| export
def untar_data(path, save_dir = None):
    # will untar a file to save dir or the directory of path
    if isinstance(path,Path):
        if str(path).endswith("tgz"):
            tar = tarfile.open(fname, "r:gz")
        else:
            tar = open(path)
        tar.extractall(save_dir) if save_dir else tar.extractall(path.parent)
        tar.close()
    else:
        print('argument should be Path instance')

SyntaxError: invalid syntax (<ipython-input-13-bd3c0ba8cb98>, line 5)

In [ ]:
#| export
logging.getLogger().setLevel(logging.INFO)
@contextlib.contextmanager
def timeit(description=None):
    # usage: with timeit('description') do what you like
    # will output the time taken in seconds
    tstart = time.time()
    yield
    elapsed = (time.time() - tstart)/60
    logging.info(f'{description}, time: {elapsed}')


In [ ]:
#| export 

def load(filename):
    f = open(filename,"rb")
    return pickle.load(f)
    
def save(data, filename):
    with open(filename, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#| export
def nan_inspect(df):
    # will return some sort or correlation matrix
    # this helps to quickly see where the na values are and if they are shared with multiple columns
    print(df.isnull().sum(axis=1).value_counts(sort=False))
    df_len = len(df)
    df = df.isna().copy()
    df = df.loc[:, (df != 0).any(axis=0)]
    df = df.loc[(df!=0).any(1)]
    df_nan_dict = {c : [len(df.loc[df[c] & df[c2]]) for c2 in df.columns] for c in df.columns}
    df_nan = pd.DataFrame.from_dict(df_nan_dict)
    df_nan.set_index(df.columns, inplace=True)
    df_nan.insert(0, 'total', df_nan.max(axis=0))
    df_nan.sort_values('total', inplace=True, ascending=False)
    df_nan = pd.concat([df_nan['total'] , df_nan.iloc[:,1:][list(df_nan.index)]],axis=1)
    return (df_nan/df_len).round(2)